## NEED TO DOWNLOAD
### 1. selenium library (pip install selenium)
### 2. Webdriver (in Library/bin/)
#### http://selenium-python.readthedocs.io/locating-elements.html#locating-elements-by-css-selectors
<br/>
#### https://seleniumhq.github.io/selenium/docs/api/py/index.html

In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
ㅇ

ImportError: No module named selenium

In [47]:
driver = webdriver.Chrome()

In [48]:
driver.get("http://marketdata.krx.co.kr/mdi#document=10020102")

In [13]:
#종목을 알고있을때
elem = driver.find_element_by_name("isu_cdnm")
elem.clear()
elem.send_keys("SAMSUNG")

In [14]:
#종목검색 창 켜기
find = driver.find_element_by_name("finderbtn")
find.click()


#정보를 모두 불러온 후
td = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.NAME, "codeName")))

#시장구분
inputs =driver.find_elements_by_css_selector("input[type='radio']")
#KOSPI
inputs[1].click()
#KOSDAQ
#inputs[2].click()
#KOSNEX
#inputs[3].click()
#채권
#inputs[4].click()


#검색어 입력
search = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.NAME, "searchText")))
search.clear()
search.send_keys("삼성화재".decode("utf-8"))

#조회
driver.find_element_by_css_selector("button.btn-board.btn-board-search").click()

#첫번째가 가장 유사한 항목.. 뜨기전까지 기다렸다가, 해당하는 a tag를 클릭
try:
    td = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.NAME, "codeName")))
    a = driver.find_element_by_link_text(td.text)
    a.click()

    #종목이 반영될 때까지 잠시 기다렸다가 모든 버튼을 긁어서 그중 조회를 클릭하고, 잠시 후에 엑셀 다운로드
    time.sleep(1)
    
    
    #날짜 설정
    calendar(driver, 3, 10)
    
    time.sleep(1)
    
    
    buttons = driver.find_elements_by_tag_name("button")
    btn_dict = dict()
    for btn in buttons:
        if btn.text:
            btn_dict[btn.text] = btn
    btn_dict["조회".decode("utf-8")].click()
    time.sleep(3)
    btn_dict["Excel"].click()
except:
    print "No data"
    
time.sleep(1)
driver.quit()

In [7]:
def calendar(driver, startdays, enddays):
    driver.find_element_by_css_selector("button.cal-btn-open").click()
    time.sleep(1)



    import datetime
    #datetime.timedelta(days=)를 바꿔가며 목표 시작 날짜 조정
    start = datetime.datetime.now() - datetime.timedelta(days=startdays)
    start_str= ("%04d년 %02d월"%(start.year, start.month)).decode("utf-8")
    default_str = driver.find_elements_by_css_selector("div.cal-header")[0].find_element_by_tag_name("span").text
    start_int = int(start_str[0:4])*100 + int(start_str[6:8])
    default_int = int(default_str[0:4])*100 + int(default_str[6:8])


    if start_int < default_int:
        button = driver.find_elements_by_css_selector("div.cal-header")[0].find_elements_by_tag_name("button")[1]
        for i in range( int((default_int - start_int) / 100) * 12 + (default_int - start_int)):
            button.click()
    elif start_int > default_int:
        button = driver.find_elements_by_css_selector("div.cal-header")[0].find_elements_by_tag_name("button")[2]
        for i in range( int((start_int - default_int) / 100) * 12 + (start_int - default_int)):
            button.click()

    time.sleep(1)
    driver.find_elements_by_css_selector("div.cal-body")[0].find_element_by_link_text(str(start.day)).click()


    #datetime.timedelta(days=)를 바꿔가며 목표 끝 날짜 조정
    end = datetime.datetime.now() + datetime.timedelta(days=enddays)
    end_str= ("%04d년 %02d월"%(end.year, end.month)).decode("utf-8")
    default_str = driver.find_elements_by_css_selector("div.cal-header")[1].find_element_by_tag_name("span").text
    end_int = int(end_str[0:4])*100 + int(end_str[6:8])
    default_int = int(default_str[0:4])*100 + int(default_str[6:8])

    if end_int < default_int:
        button = driver.find_elements_by_css_selector("div.cal-header")[1].find_element_by_css_selector("button.cal-btn-prevM")
        for i in range( int((default_int - end_int) / 100) * 12 + (default_int - end_int)):
            button.click()
            button = driver.find_elements_by_css_selector("div.cal-header")[1].find_element_by_css_selector("button.cal-btn-prevM")
    elif end_int > default_int:
        button = driver.find_elements_by_css_selector("div.cal-header")[1].find_element_by_css_selector("button.cal-btn-nextM")
        for i in range( int((end_int - default_int) / 100) * 12 + (end_int - default_int)):
            button.click()
            button = driver.find_elements_by_css_selector("div.cal-header")[1].find_element_by_css_selector("button.cal-btn-nextM")



    time.sleep(1)
    driver.find_elements_by_css_selector("div.cal-body")[1].find_element_by_link_text(str(end.day)).click()

    driver.find_element_by_css_selector("button.cal-btn-confirm").click()

In [9]:
import win32com.client
from win32com.client import constants as c
excel = win32com.client.Dispatch("Excel.Application")
excel.Visible = True
#print excel.ActiveWorkbook.FullName
wb = excel.Workbooks.Open("C:\Users\lafamila\Downloads\data.xls")

#Sheet 추가
#ws = wb.Worksheets.Add()
#ws.Name = "hello"
#활성화 되어있는 sheet
ws = wb.ActiveSheet
#특정 sheet
#ws = wb.sheets("Sheet3")


#cell 내용 가져오기
a = ws.Cells(1, 1).Value
print a
#범위 cell 지정
#ws.Range(ws.Cells(3, 1), ws.Cells(3, 4)).value = [5, 6, 7, 8]
#ws.Range("A4:D4").value = [9, 10, 11, 12]

#cell 에 함수넣기
#ws.Cells(5,4).Formula = '=SUM(A2:D4)'

#cell 속성
#ws.Cells(5,4).Font.Size = 16
#ws.Cells(5,4).Font.Bold = True

#다른이름으로 저장
#ws.SaveAs('c:\\Users\\lafamila\\Desktop\\test4.xlsx')
excel.Quit()

순위


In [ ]:
driver.find_elements_by_tag_name("textarea")[0].send_keys(random.choic(sentence))
WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.TAG_NAME, "textarea")))[0].send_keys(random.choice(sentence))
